<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#加速：非同步爬蟲" data-toc-modified-id="加速：非同步爬蟲-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>加速：非同步爬蟲</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Python-中的非同步爬蟲" data-toc-modified-id="Python-中的非同步爬蟲-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Python 中的非同步爬蟲</a></span></li><li><span><a href="#比較一下" data-toc-modified-id="比較一下-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>比較一下</a></span></li></ul></li><li><span><a href="#作業目標" data-toc-modified-id="作業目標-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>作業目標</a></span></li><li><span><a href="#莫凡教學" data-toc-modified-id="莫凡教學-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>莫凡教學</a></span><ul class="toc-item"><li><span><a href="#基本用法" data-toc-modified-id="基本用法-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>基本用法</a></span></li><li><span><a href="#有了對asyncio的基本了解,我們就來看怎麼把它用在爬蟲" data-toc-modified-id="有了對asyncio的基本了解,我們就來看怎麼把它用在爬蟲-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>有了對asyncio的基本了解,我們就來看怎麼把它用在爬蟲</a></span></li></ul></li></ul></li></ul></div>

# 加速：非同步爬蟲

* 了解非同步爬蟲加速原理與實作

### Python 中的非同步爬蟲

In [9]:
import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

async def main():
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, 'http://python.org')
        print(html)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())
# loop.close()

<!doctype html>
<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->
<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->
<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->
<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->

<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.js">
    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jqueryui/1.12.1/jquery-ui.min.js">

    <meta name="application-name" content="Python.org">
    <meta name="msapplication-tooltip" content="The official home of the Python Programming Language">
    <meta name="apple-mobile-web-app-title" content="Python.org">
    <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">

    <meta name="

### 比較一下

In [11]:
import requests, time

URL = 'https://morvanzhou.github.io/'

def normal():
    for i in range(2):
        # 發兩次 request
        r = requests.get(URL)
        url = r.url
        print(url)

t1 = time.time()
normal()
print("Normal total time:", time.time()-t1)


https://morvanzhou.github.io/
https://morvanzhou.github.io/
Normal total time: 0.5952188968658447


In [7]:
import aiohttp, asyncio
import nest_asyncio
nest_asyncio.apply()
# Optionally the specific loop that needs patching can be given as argument to apply, otherwise the current event loop is used. An event loop can be patched whether it is already running or not. Only event loops from asyncio can be patched; Loops from other projects, such as uvloop or quamash, generally can’t be patched.
# 使用 Session 登陸
# Resources: https://mofanpy.com/tutorials/data-manipulation/scraping/requests/
# 很不錯的教學:https://ithelp.ithome.com.tw/articles/10199385?sc=pt
async def job(session):
    #等待並切換
    response = await session.get(URL)                               
    return str(response.url)

async def main(loop):
    #官網推薦建立Session的形式,也可以直接用request
    # aiohttp.ClientSession()
    # Client session is the recommended interface for making HTTP requests.Session encapsulates a connection pool (connector instance) and supports keepalives by default. 
        # Resources: https://docs.aiohttp.org/en/stable/client_reference.html#aiohttp.ClientSession
    async with aiohttp.ClientSession() as session:                  
        tasks = [loop.create_task(job(session)) for _ in range(2)]
        #收集完成的結果,會返回完成的和沒完成的,等全部都完成了才返回
        finished, unfinished = await asyncio.wait(tasks)   
        #獲取所有結果
        all_results = [r.result() for r in finished]                
        print(all_results)

# 開始時間
t1 = time.time()
# event_loop 是一個 list 管家，負責切換任務的機制
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))
# loop.run_until_complete(coroutine)
# coroutine是協程的意思，但其意義在下面會介紹，目前只要把他理解成一個任務就行了
# 這個函數顧名思義，就是讓註冊參數裡的任務並執行，等到任務完成就關閉Event Loop
# loop.close()
print("Async total time:", time.time() - t1)

['https://morvanzhou.github.io/', 'https://morvanzhou.github.io/']
Async total time: 0.2406618595123291


## 作業目標

* 比較一下非同步爬蟲跟多線程爬蟲的差異是什麼？各自的優缺點為何？

## 莫凡教學
- https://mofanpy.com/tutorials/data-manipulation/scraping/asyncio/

### 基本用法

In [13]:
# 不是异步的
import time
def job(t):
    print('Start job ', t)
    time.sleep(t)               # wait for "t" seconds
    print('Job ', t, ' takes ', t, ' s')
def main():
    [job(t) for t in range(1, 3)]
t1 = time.time()
main()
print("NO async total time : ", time.time() - t1)

Start job  1
Job  1  takes  1  s
Start job  2
Job  2  takes  2  s
NO async total time :  3.0098001956939697


In [15]:
# 異步
import asyncio
async def job(t):                   # async 形式的功能
    print('Start job ', t)
    await asyncio.sleep(t)          # 等待 "t" 秒, 期间切换其他任务
    print('Job ', t, ' takes ', t, ' s')

async def main(loop):                       # async 形式的功能
    tasks = [
    loop.create_task(job(t)) for t in range(1, 3)
    ]                                       # 创建任务, 但是不执行
    await asyncio.wait(tasks)               # 执行并等待所有任务完成
    
t1 = time.time()
loop = asyncio.get_event_loop()             # 建立 loop
loop.run_until_complete(main(loop))         # 执行 loop
# loop.close()                                # 关闭 loop
print("Async total time : ", time.time() - t1)

Start job  1
Start job  2
Job  1  takes  1  s
Job  2  takes  2  s
Async total time :  2.002650737762451


### 有了對asyncio的基本了解,我們就來看怎麼把它用在爬蟲

In [25]:
# 一般的 request
import requests

URL = 'https://mofanpy.com/'
def normal():
    for i in range(2):
        r = requests.get(URL)
        url = r.url
        print(url)

t1 = time.time()
normal()
print("Normal total time:", time.time()-t1)

https://mofanpy.com/
https://mofanpy.com/
Normal total time: 2.4834001064300537


In [22]:
# 使用 aiohttp
import aiohttp
async def job(session):
    response = await session.get(URL)       # 等待并切换
    return str(response.url)


async def main(loop):
    async with aiohttp.ClientSession() as session:      # 官网推荐建立 Session 的形式
        tasks = [loop.create_task(job(session)) for _ in range(2)]
        finished, unfinished = await asyncio.wait(tasks)
        all_results = [r.result() for r in finished]    # 获取所有结果
        print(all_results)

t1 = time.time()
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))
# loop.close()
print("Async total time:", time.time() - t1)

['https://mofanpy.com/', 'https://mofanpy.com/']
Async total time: 0.3668632507324219
